In [1]:
%%writefile model.py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


class Model:
  def __init__(self):
    stk_data = pd.read_csv('stk_data.csv')
    self.data, self.test_data = train_test_split(
      stk_data, test_size=.2, shuffle=False)
    self.scaler = MinMaxScaler(feature_range=(0,1))
    self.scaled_data = self.scaler.fit_transform(
      self.data['Close'].values.reshape(-1,1))
    prediction_days = 60
    self.x_train = []
    self.y_train = []
    for x in range(prediction_days, len(self.scaled_data)):
      self.x_train.append(self.scaled_data[x-prediction_days:x, 0])
      self.y_train.append(self.scaled_data[x, 0])
    self.x_train, self.y_train = np.array(self.x_train), np.array(self.y_train)
    self.x_train = np.reshape(
      self.x_train, (self.x_train.shape[0], self.x_train.shape[1], 1))
    #test data preparation
    self.actual_prices = self.test_data['Close'].values
    self.total_dataset = pd.concat(
      (self.data['Close'], self.test_data['Close']), axis=0)
    self.model_inputs = self.total_dataset[
      len(self.total_dataset) - len(self.test_data) - prediction_days:].values
    self.model_inputs = self.model_inputs.reshape(-1,1)
    self.model_inputs = self.scaler.transform(self.model_inputs)
    self.x_test = []
    for x in range(prediction_days, len(self.model_inputs)):
      self.x_test.append(self.model_inputs[x-prediction_days:x, 0])
    self.x_test = np.array(self.x_test)
    self.x_test = np.reshape(
      self.x_test, (self.x_test.shape[0], self.x_test.shape[1], 1))
    return

  def create(self):
    self.model = Sequential()
    self.model.add(LSTM(units=50, return_sequences=True, input_shape=(
      self.x_train.shape[1], 1)))
    self.model.add(Dropout(0.2))
    self.model.add(LSTM(units=50, return_sequences=True))
    self.model.add(Dropout(0.2))
    self.model.add(LSTM(units=50))
    self.model.add(Dropout(0.2))
    self.model.add(Dense(units=1))
    self.model.compile(optimizer='adam', loss='mean_squared_error')
    self.model.fit(self.x_train, self.y_train, epochs=25, batch_size=32)
    return self.model
    
  def predict(self, model):
    self.model = model
    self.predicted_prices = self.model.predict(self.x_test)
    self.predicted_prices = self.scaler.inverse_transform(self.predicted_prices)

  def plotting(self):
    plt.plot(self.actual_prices, color = "black", label=f"Actual {company} price")
    plt.plot(self.predicted_prices, color='green', label=f'Predicted {company} prices')
    plt.title(f"{company} Share Price")
    plt.xlabel('Time')
    plt.ylabel(f'{company} Share Price')
    plt.legend()
    plt.show()

  def results(self):
    prediction_days = 60
    self.real_data = [
      self.model_inputs[
        len(self.model_inputs)+1-prediction_days:len(self.model_inputs+1), 0]]
    self.real_data = np.array(self.real_data)
    self.real_data = np.reshape(
      self.real_data, (self.real_data.shape[0], self.real_data.shape[1], 1))
    prediction = self.model.predict(self.real_data)
    prediction = self.scaler.inverse_transform(prediction)
    r2 = r2_score(self.actual_prices, self.predicted_prices)
    return {'r2': r2,'prediction': prediction}


Writing model.py


In [2]:
import pandas_datareader as web
import datetime as dt

In [16]:
def save(tik):
  data = web.DataReader(tik, 'yahoo', start, end)
  data.to_csv('stk_data.csv', index=True)
  nn = Model()
  seq = nn.create()
  seq.save(f'models/{tik}_nn')

In [33]:
tiks = [
        'TTC',
        'CCIV',
        'PLUG',
        'Z74.SI',
        'FCEL',
]

In [34]:
for i in tiks:
  save(i)

Epoch 1/25
57/57 [==============================] - 9s 80ms/step - loss: 0.0747
Epoch 2/25
57/57 [==============================] - 4s 79ms/step - loss: 0.0033
Epoch 3/25
57/57 [==============================] - 5s 79ms/step - loss: 0.0032
Epoch 4/25
57/57 [==============================] - 5s 80ms/step - loss: 0.0031
Epoch 5/25
57/57 [==============================] - 5s 81ms/step - loss: 0.0029
Epoch 6/25
57/57 [==============================] - 5s 82ms/step - loss: 0.0028
Epoch 7/25
57/57 [==============================] - 5s 81ms/step - loss: 0.0028
Epoch 8/25
57/57 [==============================] - 5s 84ms/step - loss: 0.0032
Epoch 9/25
57/57 [==============================] - 5s 84ms/step - loss: 0.0024
Epoch 10/25
57/57 [==============================] - 5s 82ms/step - loss: 0.0028
Epoch 11/25
57/57 [==============================] - 5s 83ms/step - loss: 0.0028
Epoch 12/25
57/57 [==============================] - 5s 83ms/step - loss: 0.0027
Epoch 13/25
57/57 [==================

INFO:tensorflow:Assets written to: models/TTC_nn/assets


INFO:tensorflow:Assets written to: models/TTC_nn/assets


Epoch 1/25
2/2 [==============================] - 5s 80ms/step - loss: 0.1599
Epoch 2/25
2/2 [==============================] - 0s 73ms/step - loss: 0.0982
Epoch 3/25
2/2 [==============================] - 0s 90ms/step - loss: 0.0689
Epoch 4/25
2/2 [==============================] - 0s 75ms/step - loss: 0.0423
Epoch 5/25
2/2 [==============================] - 0s 80ms/step - loss: 0.0718
Epoch 6/25
2/2 [==============================] - 0s 77ms/step - loss: 0.0485
Epoch 7/25
2/2 [==============================] - 0s 78ms/step - loss: 0.0396
Epoch 8/25
2/2 [==============================] - 0s 81ms/step - loss: 0.0467
Epoch 9/25
2/2 [==============================] - 0s 74ms/step - loss: 0.0469
Epoch 10/25
2/2 [==============================] - 0s 77ms/step - loss: 0.0457
Epoch 11/25
2/2 [==============================] - 0s 75ms/step - loss: 0.0417
Epoch 12/25
2/2 [==============================] - 0s 78ms/step - loss: 0.0411
Epoch 13/25
2/2 [==============================] - 0s 77ms/st

INFO:tensorflow:Assets written to: models/CCIV_nn/assets


...

INFO:tensorflow:Assets written to: models/Z74.SI_nn/assets


INFO:tensorflow:Assets written to: models/Z74.SI_nn/assets


Epoch 1/25
57/57 [==============================] - 9s 80ms/step - loss: 0.0181
Epoch 2/25
57/57 [==============================] - 5s 82ms/step - loss: 0.0024
Epoch 3/25
57/57 [==============================] - 5s 83ms/step - loss: 0.0020
Epoch 4/25
57/57 [==============================] - 5s 86ms/step - loss: 0.0019
Epoch 5/25
57/57 [==============================] - 5s 86ms/step - loss: 0.0018
Epoch 6/25
57/57 [==============================] - 5s 85ms/step - loss: 0.0024
Epoch 7/25
57/57 [==============================] - 5s 87ms/step - loss: 0.0016
Epoch 8/25
57/57 [==============================] - 5s 89ms/step - loss: 0.0016
Epoch 9/25
57/57 [==============================] - 5s 88ms/step - loss: 0.0016
Epoch 10/25
57/57 [==============================] - 5s 91ms/step - loss: 0.0014
Epoch 11/25
57/57 [==============================] - 5s 91ms/step - loss: 0.0016
Epoch 12/25
57/57 [==============================] - 5s 88ms/step - loss: 0.0012
Epoch 13/25
57/57 [==================

INFO:tensorflow:Assets written to: models/FCEL_nn/assets


INFO:tensorflow:Assets written to: models/FCEL_nn/assets


In [22]:
company = 'RELI'

In [23]:
start = dt.datetime(2012,1,1)
end = dt.datetime.now()

In [24]:
data = web.DataReader(company, 'yahoo', start, end)
data.to_csv('stk_data.csv',index=True)

In [25]:
from model import Model

In [ ]:
nn = Model()

In [15]:
seq = nn.create()
seq.save(f'models/{company}_nn')

Epoch 1/25
57/57 [==============================] - 9s 77ms/step - loss: 0.0657
Epoch 2/25
57/57 [==============================] - 4s 78ms/step - loss: 0.0037
Epoch 3/25
57/57 [==============================] - 5s 81ms/step - loss: 0.0028
Epoch 4/25
57/57 [==============================] - 5s 85ms/step - loss: 0.0029
Epoch 5/25
57/57 [==============================] - 5s 84ms/step - loss: 0.0029
Epoch 6/25
57/57 [==============================] - 5s 87ms/step - loss: 0.0029
Epoch 7/25
57/57 [==============================] - 5s 89ms/step - loss: 0.0026
Epoch 8/25
57/57 [==============================] - 5s 86ms/step - loss: 0.0023
Epoch 9/25
57/57 [==============================] - 5s 87ms/step - loss: 0.0023
Epoch 10/25
57/57 [==============================] - 5s 85ms/step - loss: 0.0021
Epoch 11/25
57/57 [==============================] - 5s 83ms/step - loss: 0.0023
Epoch 12/25
57/57 [==============================] - 5s 86ms/step - loss: 0.0018
Epoch 13/25
57/57 [==================

INFO:tensorflow:Assets written to: models/AAPL_nn/assets


INFO:tensorflow:Assets written to: models/AAPL_nn/assets


In [9]:
from tensorflow import keras
model = keras.models.load_model('stock_smart')

In [11]:
nn.predict(model)

In [12]:
nn.results()

{'prediction': array([[290.49545]], dtype=float32), 'r2': 0.9081610035271611}

In [35]:
!zip -r ./models.zip ./models

  adding: models/ (stored 0%)
  adding: models/FCEL_nn/ (stored 0%)
  adding: models/FCEL_nn/assets/ (stored 0%)
  adding: models/FCEL_nn/saved_model.pb (deflated 90%)
  adding: models/FCEL_nn/variables/ (stored 0%)
  adding: models/FCEL_nn/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: models/FCEL_nn/variables/variables.index (deflated 67%)
  adding: models/TTC_nn/ (stored 0%)
  adding: models/TTC_nn/assets/ (stored 0%)
  adding: models/TTC_nn/saved_model.pb (deflated 90%)
  adding: models/TTC_nn/variables/ (stored 0%)
  adding: models/TTC_nn/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: models/TTC_nn/variables/variables.index (deflated 67%)
  adding: models/DOGE-USD_nn/ (stored 0%)
  adding: models/DOGE-USD_nn/assets/ (stored 0%)
  adding: models/DOGE-USD_nn/saved_model.pb (deflated 90%)
  adding: models/DOGE-USD_nn/variables/ (stored 0%)
  adding: models/DOGE-USD_nn/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: models/DOGE-USD_nn/